In [163]:
import torch
import numpy as np

## Loss functions and tensor types

The tensor types matter

In [29]:
loss = torch.nn.CrossEntropyLoss()
x    = torch.tensor([[0.2, 0.3, 0.4],[2, 0.3,0.6]])
y    = torch.tensor([0.,1.])

#  Uncomment line (*) and see: 
## RuntimeError: Expected object of scalar type Long but got scalar type Double for argument #2 'target'

#  loss(x,y) # (*) Uncommenting test

Nevertheless the following tensor will work

In [117]:
?torch.nn.CrossEntropyLoss

In [150]:
loss   = torch.nn.CrossEntropyLoss()
x      = torch.tensor([[0.2, 0.3, 0.4],[2, 0.3,0.6]], requires_grad=True)
y      = torch.tensor([0,1])
weigth = loss(x,y)
output = weigth*loss(x,y)

In [164]:
output

tensor(2.6555, grad_fn=<MulBackward0>)

### Implement a modification of a loss function

In [156]:
import warnings

from torch import functional as F
from torch.nn import Module
from torch.nn import _reduction as _Reduction
from torch._jit_internal import weak_module, weak_script_method
from torch.nn.functional import cross_entropy

class _Loss(Module):
    def __init__(self, size_average=None, reduce=None, reduction='mean'):
        super(_Loss, self).__init__()
        if size_average is not None or reduce is not None:
            self.reduction = _Reduction.legacy_get_string(size_average, reduce)
        else:
            self.reduction = reduction

class _WeightedLoss(_Loss):
    def __init__(self, weight=None, size_average=None, reduce=None, reduction='mean'):
        super(_WeightedLoss, self).__init__(size_average, reduce, reduction)
        self.register_buffer('weight', weight)
        

class CrossEntropyLossSquared(_WeightedLoss):

    def __init__(self, weight=None, size_average=None, ignore_index=-100,
                 reduce=None, reduction='mean'):
        super(CrossEntropyLossSquared, self).__init__(weight, size_average, reduce, reduction)
        self.ignore_index = ignore_index

    @weak_script_method
    def forward(self, input, target):
        return cross_entropy(input, target, weight=self.weight,
                               ignore_index=self.ignore_index, reduction=self.reduction)**2



In [157]:
loss = CrossEntropyLossSquared()
x    = torch.tensor([[0.2, 0.3, 0.4],[2, 0.3,0.6]])
y    = torch.tensor([0,1])
out  = loss(x,y)

In [158]:
out

tensor(2.6555)